### This jupyter notebook cell initializes an OpenHSIServer object with three rosservices:

#### /update6S

Recalculates the predicted incident radiation using the 6s model and then saves it to the calibration .pkl file. The 6s model obtains weather balloon data from the radiosonde station specified in the settings .json. For example the radiosonde_region "TBW" and station number 72210 correspond to the weather balloon station in the Tampa Bay area. Weather balloon stations launch balloons twice a day, at 6am and 6pm. This service is relevant because incident radiation should change throughout the day. I am not totally sure, but I think the lat and long in the settings are supposed to be the coordinates of the hs camera.

#### /take_image IMAGE_DIR

Makes the hs camera take an image, process to reflectivity, and save the .png and datacube file in directory IMAGE_DIR. Once the camera is connected to your computer via an 802.3af compliant POE injector, the connection is indicated by the camera's red LED changing to green. When the LED is green, a new LucidCamera object should be able to detect the camera. It may require some fiddling with your network settings for wired connections to get the connection working. For example, entering the MAC address of the ethernet port you are trying to use might be necessary. OpenHSI also recommends using an MTU size of 9000 for the connection for taking images faster.

#### /compare_images IMAGE_DIR1 IMAGE_DIR2

Compares the reflectivity spectra in the datacubes in IMAGE_DIR1 and IMAGE_DIR2. Currently it selects a grid of pixels across the image, and then averages the reflectivity values of each wavelength across all of the sampled pixels. Then it calculates the euclidean distance between the two spectra. The lower the distance, the closer the two spectra are together and more likely they are from the same object.


#### Troubleshooting:

- To call a service, have roscore running, then type command "rosservice call /SERVICE_NAME  [PARAM1]  [PARAM2]" in another terminal.

- If you keep getting negative reflectances or weird values that are obviously wrong, try changing the z_time (zulu time) parameter and checking the model's predicted radiance. Reflectance is calculated as reflected_radiation / incident_radiation, so if the predicted incident radiation is close to zero because it is nighttime, then the reflectance values may be off.

#### Links:

- For tutorials and info about the OpenHSI project, their github page is here: https://openhsi.github.io/

- Reflectance profiles that we will have to detect for the 2022 wildlife challenge: https://robotx.org/programs/robotx-challenge-2022/#resources

- Technical document about the camera: https://www.dropbox.com/s/3akr8s9h111yi2i/OpenHSI_Tech_summary.pdf?dl=0

- Calibration files for our camera, serial number OpenHSI-18: https://www.dropbox.com/sh/77lrkw3e6ntburh/AABz9hlOI3kNcSz_vhOSi5TFa?dl=0

- 2022 RobotX Challenge Forums (there were some posts about the hs cam there): https://robonationforum.vbulletin.net/forum/robotx/-2022-robotx-challenge

- RoboNation community discord: https://discord.gg/BRHh6VMCJp

In [ ]:
#!/usr/bin/env python3
import os
import numpy as np
import json
from urllib.error import *
from datetime import datetime
from Py6S import *
from scipy.interpolate import interp1d
import rospy
from mil_openhsi.srv import *
from openhsi_deps.cameras import *
from openhsi_deps.data import *
from openhsi_deps.atmos import *
from openhsi_deps.capture import *
from openhsi_deps.calibrate import *

PATH = os.getenv('MIL_WS')

JSON_PATH = PATH + "/src/mil/InvestiGator/perception/mil_openhsi/OpenHSI-18_settings_Mono8_bin2.json"
PKL_PATH  = PATH + "/src/mil/InvestiGator/perception/mil_openhsi/OpenHSI-18_calibration_Mono8_bin2.pkl"

cam_props = CameraProperties(JSON_PATH, PKL_PATH)

class OpenHSIServer:        
    def __init__(self):
        self.update_service = rospy.Service("/update6S", update6S, self.update6S)
        self.reset_service = rospy.Service("/take_image", specSig, self.take_image)
        self.compare_service = rospy.Service("/compare_images", compareImgs, self.compare_images)
        
        self.model = None
        self.update6S()
        
        
    #updates 6S because the predicted incident radiance should change throughout the day
    def update6S(self, req=None):
        while True:
            try:
                self.model = Model6SV(lat = cam_props.settings["latitude"], lon = cam_props.settings["longitude"],
                    #z_time = datetime.strptime(cam_props.settings["datetime_str"], "%Y-%m-%d %H:%M"),
                    z_time = datetime.now(),
                    station_num = cam_props.settings["radiosonde_station_num"],region = cam_props.settings["radiosonde_region"],
                    alt = cam_props.settings["altitude"], zen = 0, azi = 0,
                    aero_profile = AeroProfile.Maritime,
                    wavelength_array = np.linspace(350,900,num=130),
                    sixs_path = cam_props.settings["sixs_path"])
            except HTTPError:
                print("Error connecting to radiosonde station server, retrying...")
            else:
                break
        
        print("model radiance at", datetime.now(), ": \n")
        #the 6s model computes incident radiance in units of (μW/cm^2/sr/nm), which are different units from the reflected radiance (W/m^2/sr/μm)
        #computed from the hs camera's digital values, which is why there is a divide by 10
        #wavelengths measured range from 350nm to 900nm with 130 steps between
        cam_props.calibration["rad_fit"] = interp1d(np.linspace(350,900,num=130), self.model.radiance/10, kind='cubic')
        print(self.model.radiance/10)
        cam_props.dump(JSON_PATH, PKL_PATH)
            
        
    def take_image(self, req):
        print(req.img_name)
        
        with LucidCamera(n_lines        = 100,
                processing_lvl = 6,
                pkl_path       = PKL_PATH,
                json_path      = JSON_PATH,
                exposure_ms    = 5.999864
               ) as cam:
            
            cam.collect()
            print(type(cam.dc))
            print("size0: ")
            print(cam.dc.size[0])
            print("size1: ", cam.dc.size[1])
            print("size2: ", cam.dc.size[2])

            step = np.linspace(350,900, num=130, retstep=True)
            cam.save(req.img_name)
            cam.show(robust=True)
    

    #loads and compares two images
    def compare_images(self, req):
        def _euclid_dist(pix1, pix2):
            distance = 0
            for i in range(len(pix1)):
            #each pix[i] represents the reflectivity for wavelength (i * (900 - 350)/130 + 350) nm
                distance += (pix1[i] - pix2[i]) * (pix1[i] - pix2[i])
            return np.sqrt(distance)
                
        def sample_pixels(dc:DataCube):
            #dc.dc is a CircArrayBuffer of shape [cross_track(height), along_track(width), wavelength_index] = intensity
            #refer to https://openhsi.github.io/openhsi/api/data.html for more info on the datatypes in OpenHSI
            print("size of dc.dc: ", dc.dc.size)
            height = dc.dc.size[0]
            width = dc.dc.size[1]
            
            #step is the number of pixels in a column or row in the N x N grid of sampled pixels
            N = 10
            
            #add one to step to account for the edge row, which is to be ignored
            cross_step = round(height/(N + 1))
            along_step = round(width/(N + 1))
            
            cross = cross_step
            along = along_step
            
            #plan is to add cross_step N number of times to sample a grid of pixels with none from the edges of the image,
            #since the edges of the image may have areas where there is no data/all values are zero
            i = 0
            j = 0
            wav_index = 0
            avg_intensities = [0] * dc.dc.size[2]
        
            while i < N:
                j = 0
                along = along_step
                while j < N:
                    wav_index = 0
                    while wav_index < dc.dc.size[2]:
                        avg_intensities[wav_index] += dc.dc[cross][along][wav_index] / (N*N)
                        wav_index += 1
                    along += along_step
                    j += 1
                cross += cross_step
                i += 1
            return avg_intensities
        
        #load the two images into datacube objects
        image1 = DataCube(processing_lvl=6,json_path=JSON_PATH,pkl_path=PKL_PATH)
        image2 = DataCube(processing_lvl=6,json_path=JSON_PATH,pkl_path=PKL_PATH)
        
        image1.load_nc(nc_path = req.img1_dir + "/scan2.nc")
        image2.load_nc(nc_path = req.img2_dir + "/scan2.nc")
        
        #sample pixels at regular intervals for each image
        #average the intensities for each particular wavelength over all sampled pixels
        avg_pix1 = sample_pixels(image1)
        avg_pix2 = sample_pixels(image2)
        
        #compute the euclidean distance between the two images' average spectra
        distance = _euclid_dist(avg_pix1, avg_pix2)
        
        #to compare two pixels, run the following
        # distance = _euclid_dist(image1.dc[200][50], image2.dc[200][50])        
        
        #return the euclidean distance
        return distance

if __name__ == '__main__':
    rospy.init_node('openhsi_server')
    OpenHSIServer()
    rospy.spin()

/home/mitch/miniforge3/lib/python3.10/site-packages/pysolar/solartime.py:111: UserWarning: I don't know about leap seconds after 2021
  warnings.warn \
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 130/130 [00:06<00:00, 20.93it/s]


model radiance at 2023-04-29 16:00:16.300475 : 

[29.2741     31.1477     29.7443     31.3779     35.1975     33.524
 31.0798     34.7205     27.746      31.1957     31.3227     38.4608
 45.78262045 49.9343     51.6678     53.955      52.9168     51.5063
 48.2195     45.3094     53.0635     53.9523     58.1061     60.0011
 61.3262     60.9921     61.2362     60.3697     59.5707     60.0165
 60.9343     58.7112     56.8207     57.6227     58.7525     56.259
 56.7375     57.2213     55.5288     53.4402     53.0444     54.5687
 55.8229     55.5518     54.5973     54.1972     54.5056     54.76199381
 54.0665     52.5439     52.7485     52.7081     52.6271     52.6178
 52.439      52.6275     49.11199926 46.7597     49.5095     50.1456
 50.3205     49.4895     48.2117     48.6672     48.5585     47.3906
 47.0563     47.4387     47.215      46.499      44.4348     43.5416
 42.8175     44.6625     45.2829     44.7282     44.3152     44.0862
 43.4012     33.0615     39.3273     35.6919     37.